In [ ]:
import os
import sys
sys.path.append(os.path.abspath('..'))
from config import *

!cp ../data/acorns.clean.csv .

# 線形回帰

```{index} せ 線形回帰
:name: 線形回帰
```

```{index} せ 説明変数
:name: 説明変数
```

```{index} も 目的変数
:name: 目的変数
```



**回帰**（**regression**）とは、ある変数（結果）が別の変数（原因）に影響されて変化する関係を探る手法です。この関係を数式で表現し、原因と結果の結びつきを定量的に分析したり、将来を予測したりする方法を**回帰分析**（**regression analysis**）と呼びます。たとえば、天候や土壌条件が作物の収量に与える影響を分析したり、気象条件から植物病害虫の発生リスクや農作物の市場価格を予測したりする際に活用されます。

回帰には、**線形回帰**（**linear regression**）と非線形回帰がありますが、生物学や農学の分野では多くの場合、線形回帰が用いられます。回帰分析において、原因となる変数を**説明変数**（**explanatory variable**）、結果となる変数を**目的変数**（**objective variable**）と呼びます。説明変数を $X$、目的変数を $Y$ としたとき、線形回帰は次のような式で表されます。

$$
Y = \beta_{0} + \beta_{1}X + \epsilon 
$$

ここで、$\beta_{0}$ は切片、$\beta_{1}$ は説明変数に対する回帰係数、$\epsilon$ は誤差項です。

また、目的変数に影響を与える要因が複数あると考えられる場合は、$n$ 個の説明変数を用いて次のように表現します。

$$
Y = \beta_{0} + \beta_{1}X_{1} + \beta_{2}X_{2} + \cdots + \beta_{n}X_{n} + \epsilon 
$$


このとき、$\beta_{1}, \beta_{2}, \cdots, \beta_{n}$ はそれぞれの説明変数に対応する回帰係数です。実験や調査で得られた $X$ および $Y$ のデータをもとに、これらの係数を推定します。推定された回帰係数を用いて得られる式（例：$y = 1.2 + 1.6x_{1} + 0.8x_{2} + \cdots + 0.2x_{n}$）を、回帰直線、回帰モデル、あるいは単にモデルと呼びます。

線形回帰のうち、説明変数が 1 つだけの場合を**線形単回帰**（**simple linear regression**）、複数ある場合を**線形重回帰**（**multiple linear regression**）と呼びます。ただし、実用上、単回帰が使用される場面は少なく、実際には複数の説明変数を用いる重回帰が一般的です。そのため、一般に「回帰」や「回帰分析」と言えば、重回帰と考えて差し支えありません。

また、回帰モデルでは誤差項が正規分布に従うことが前提とされます。したがって、モデル構築後には、モデルによる予測値と実測値の誤差を可視化し、その誤差の分布が適切かどうかを確認する必要があります。

Python を用いて回帰分析を行う際には、主に statsmodels と scikit-learn の 2 つのライブラリが利用されます。statsmodels は統計的解析に適しており、回帰係数の推定、パラメータに対する検定、予測区間や信頼区間の算出などが可能です。一方、scikit-learn は回帰分析を機械学習の一手法として扱い、予測に重点を置いたモデリングに適しています。本節では、推定されたモデルを評価したり、信頼区間の確認したりするなどの統計的な観点を重視するため、statsmodels を使用して解説を進めます。

```{index} た 単回帰
:name: 単回帰
```

(simple_linear_regression)=
## 単回帰

どんぐりの高さ（height）を使ってその重さ（weight）を予測する回帰モデルを作成してみましょう。まず、Pandas を使ってどんぐりのデータを読み込みます。どんぐりは種類によって形が異なり、例えばクヌギは丸く、マテバシイは細長い形状をしています。そのため、すべての種類のデータをまとめて扱うと、形状の違いによってモデルの精度が下がる可能性があります。そこで今回は、クヌギのどんぐりのデータだけを抽出し、分析対象とします。モデルの作成には重さと高さの情報だけが必要なので、これら 2 列だけを取り出します。

In [ ]:
# !wget https://py.biopapyrus.jp/data/acorns.clean.csv
acorns_data = pd.read_csv('acorns.clean.csv')
kunugi = acorns_data.loc[acorns_data['tree'] == 'kunugi', ['weight', 'height']]
kunugi

次に、statsmodels ライブラリの api モジュールが提供する `OLS` 関数を使って回帰係数を推定してみましょう。`OLS` 関数では、目的変数と説明変数をそれぞれ引数として与えます。このとき、回帰モデルに定数項（$\beta_0$）を含めるために、`add_constant` 関数を使って説明変数に定数列（すべての値が 1）を追加する必要があります。以下は、クヌギのどんぐりデータを用いて回帰モデルを構築する例です。

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

X = kunugi['height']
y = kunugi['weight']

model = sm.OLS(y, sm.add_constant(X))
results = model.fit()

推定された回帰モデルの結果は、`results` オブジェクトに保存されます。モデルの詳細な概要を確認するには、次のように `summary()` メソッドを使用します。

In [ ]:
results.summary()

また、推定された回帰係数を確認したい場合は、次のようにします。

In [ ]:
results.params

In [ ]:
glue('lm_intercept', abs(round(float(results.params[0]), 2)))
glue('lm_slope', round(float(results.params[1]), 2))

今回構築した回帰モデルでは、説明変数はどんぐりの「高さ」のみです。このため、回帰式は weight = {glue:}`lm_slope`height - {glue:}`lm_intercept` のような一次式となります。

この回帰式をもとに、観測データと回帰直線を重ねてグラフで可視化してみましょう。回帰直線を描くためには、説明変数（どんぐりの高さ）と、それに対応する予測値のペアが少なくとも 2 組必要です。ここでは、回帰モデルを構築する際に使用したデータを基に、どんぐりの高さの最小値と最大値をモデルに入力し、それぞれの予測値を算出します。続けて、これら 2 点を結ぶことで回帰直線を描画します。

以下のコードでは、モデルの `predict` メソッドを用いて予測値を算出し、`plot` メソッドを使って回帰直線を描いています。さらに、測定値は `scatter` メソッドを用いて、同じグラフ上に散布図として重ねて表示します。

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()

# observed data
ax.scatter(X, y)

# regression line
x_ = [X.min(), X.max()]
y_ = results.predict(sm.add_constant(x_))
ax.plot(x_, y_)

ax.set_xlabel('height')
ax.set_ylabel('weight')
plt.show()

モデルの妥当性を検討するためには、予測値と観測値の差である残差のパターンを確認することが重要です。残差が説明変数の値に依存せず、正規分布している場合、モデルは妥当である可能性が高くなります。以下のコードは、各サンプルの誤差を視覚的に示しています。

In [ ]:
err = y - results.predict(sm.add_constant(X))

fig = plt.figure()

ax1 = fig.add_subplot(1, 2, 1)
ax1.plot([X.min(), X.max()], [0, 0])
ax1.vlines(X, 0, err, linestyles='dashed')
ax1.scatter(X, err)
ax1.set_xlabel('height')
ax1.set_ylabel('error')

ax2 = fig.add_subplot(1, 2, 2)
ax2.hist(err)

plt.show()

予測誤差の大きいどんぐりが含まれるものの、予測誤差の分布は正規分布に近いため、モデルとしては妥当と考えられれます。

回帰モデルは、分析に用いたどんぐりのデータセットに基づいて推定されたものです。そのため、同じ条件でデータを再び収集したとしても、得られる回帰直線は毎回少しずつ異なる可能性があります。仮に、同様の実験を無数に繰り返したとすると、無数の異なる回帰直線が描かれることになります。ここで、もしどんぐりの重さと高さの間に真の比例関係が存在するならば、これらの回帰直線の傾きや切片は、ある特定の値に近づいて収束し、回帰直線もある範囲内に収まると考えられます。このような無数の回帰直線のうち、95% が通ると予想される範囲を 95% **信頼区間**（**confidence interval**; **CI**）と呼びます。

信頼区間に似た概念として、**予測区間**（**prediction interval**; **PI**）があります。信頼区間が再実験によって得られる回帰直線が通る範囲を示すのに対し、予測区間は再実験で得られる新たなデータ点が存在する範囲を示します。

信頼区間および予測区間を計算するには、モデルの `get_prediction` メソッドに説明変数の値を代入します。これにより、それぞれの説明変数に対応する信頼区間および予測区間が計算されます。計算結果は、`summary_frame` メソッドを用いることで、データフレームとして取得できます。

なお、信頼区間や予測区間の幅は一定ではなく、説明変数の値に応じて変化します。そのため、信頼区間や予測区間をより正確に把握するには、説明変数の値を複数用意する必要があります。ここでは、モデル構築に使用したデータの最小値から最大値までを 100 等分し、その値を説明変数としてモデルに代入して予測を行います。

In [ ]:
x_ = np.linspace(X.min(), X.max(), 100)
y_ = results.get_prediction(sm.add_constant(x_))

y_sum = y_.summary_frame(alpha=0.05)
y_sum.head()

次に、信頼区間と予測区間をグラフに描画してみます。

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()

# data
ax.scatter(X, y)

# regression line
ax.plot(x_, y_sum['mean'])

# prediction interval
ax.fill_between(x_, y_sum['obs_ci_lower'], y_sum['obs_ci_upper'],
                lw=0, color='#426e86', alpha=0.2, label='PI 95%')

# confidence interval
ax.fill_between(x_, y_sum['mean_ci_lower'], y_sum['mean_ci_upper'],
                lw=0, color='#34675c', alpha=0.4, label='CI 95%')

ax.set_xlabel('height')
ax.set_ylabel('weight')
ax.legend()
plt.show()

```{admonition} 練習問題 SLM-2
どんぐりデータセット（acorns.clean.txt）を読み込み、各種どんぐりに対して、高さ（height）で重さ（weight）を説明する単回帰モデルを構築し、どんぐりの形と推定される回帰直線の傾きの間にどのような関係があると思われるのかを簡潔に答えよ。
```

## 重回帰

説明変数が複数ある場合の回帰分析を見ていきましょう。クヌギのどんぐりの重さ（weight）を推定するために、どんぐりの高さ（height）と直径（diameter）の 2 つの説明変数を使います。まず、解析対象のデータを抽出します。

In [ ]:
acorns_data = pd.read_csv('acorns.clean.csv')
X = acorns_data.loc[acorns_data['tree'] == 'kunugi', ['height', 'diameter']]
y = acorns_data['weight'][acorns_data['tree'] == 'kunugi']

説明変数が複数ある場合、各変数の単位や値の範囲が異なることが多いため、そのままで回帰係数を推定すると、どの説明変数が目的変数に強く影響を与えているのかを適切に分析することができません。そこで、回帰を行う前に、すべての説明変数が平均 0、分散 1 となるように**標準化**（**standardization**）を行います。標準化を行うことで、各説明変数が同じスケールで評価され、モデルの解釈がしやすくなります。この場合、`weight` 列以外のすべての列に対して正規化を行います。

In [ ]:
X = (X - X.mean()) / X.std()
X.describe()



続けて、`OLS` 関数に目的変数、定数列を追加した説明変数を順に与えて、モデルを構築します。

In [ ]:
model = sm.OLS(y, sm.add_constant(X))
results = model.fit()
results.summary()

In [ ]:
results.params

In [ ]:
glue('lm_x0', round(float(results.params[0]), 3))
glue('lm_x1', round(float(results.params[1]), 3))
glue('lm_x2', round(float(results.params[2]), 3))

回帰式は　weight = {glue}`lm_x0` + {glue}`lm_x1`height + {glue}`lm_x2`diameter のように推定されました。係数の大きさから、クヌギのどんぐりの重さを説明するには、高さよりも直径の方がより有効であると考えられます。

次に、細長い形状を持つコナラのどんぐりに対して、同様の回帰モデルを構築して分析してみよう。

In [ ]:
X_konara = acorns_data.loc[acorns_data['tree'] == 'konara', ['height', 'diameter']]
X_konara = (X_konara - X_konara.mean()) / X_konara.std()

y_konara = acorns_data['weight'][acorns_data['tree'] == 'konara']

model_konara = sm.OLS(y_konara, sm.add_constant(X_konara))
results_konara = model_konara.fit()

results_konara.summary()

結果として、高さにかかる回帰係数が直径よりも大きいことが確認されました。つまり、コナラのどんぐりの場合は、重さに対して高さがより強い影響を与えていると考えられます。

## 変数選択

回帰分析において、モデルに複数の説明変数を含めることは可能ですが、変数が多くなればなるほど必ずしも良いモデルが得られるわけではありません。過剰な変数の追加は、モデルが**過剰適合**（**overfitting**）する原因となることもあります。したがって、回帰分析を行う際には、複数の変数を様々な組み合わせでモデル化し、その中から最適なモデルを選択する作業が必要です。

統計分野でモデルの良さを測る指標として一般的に使用されているのが**赤池情報量基準**（**Akaike's information criterion**; **AIC**）です。AIC は、モデルの適合性と複雑さ（変数の数）とのバランスを取るために使われます。AIC が小さいほど、より良いモデルとされます。

クヌギのデータを利用して、どんぐりの高さのみを使って重さを説明するモデルを構築し、その AIC を計算します。

In [ ]:
acorns_data = pd.read_csv('acorns.clean.csv')
kunugi = acorns_data.loc[acorns_data['tree'] == 'kunugi', ]

X = kunugi['height']
y = kunugi['weight']

m1 = sm.OLS(y, sm.add_constant(X))
r1 = m1.fit()
r1.aic

次に、重さを直径で説明するモデル、そして重さを高さと直径の両方を使って説明するモデルの AIC も計算します。

In [ ]:
X = kunugi['diameter']
y = kunugi['weight']

m2 = sm.OLS(y, sm.add_constant(X))
r2 = m2.fit()
r2.aic

In [ ]:
X = kunugi[['height', 'diameter']]
X = (X - X.mean()) / X.std()
y = kunugi['weight']

m3 = sm.OLS(y, sm.add_constant(X))
r3 = m3.fit()
r3.aic

これらの結果を比較すると、最も AIC が低いのは、高さと直径の両方を説明変数としたモデルであることがわかります。つまり、このデータセットにおいては、どんぐりの重さを説明するためには、高さと直径の両方を含めたモデルが最適であると言えます。実際、どんぐりは一般に大きくなるほど重くなると推察され、高さおよび直径が大きさを反映しているとすれば、この結果は統計的にも直感的にも妥当と言えるでしょう。

もっとも、今回の分析に使ったデータは、筆者が虫喰いどんぐりを丁寧に取り除き、さらに「外れ値」という名の不都合な真実もそっと隠し持ちつつ測定しています。教材だからできる特権ですが、現実のどんぐりはそう簡単に協力してくれません。現実のどんぐりたちは、もっと複雑なドラマを繰り広げているに違いありません。


```{index} も-モデル選択
:name: モデル選択
```

```{index} へ-変数選択
:name: 変数選択
```

このように最適なモデルを選ぶ過程は、**モデル選択**（**model selection**）または**変数選択**（**variable selection**）と呼ばれ、回帰分析において重要なステップです。上記の例では、考えられる説明変数の組み合わせすべてで回帰モデルを作成し、最適なモデルを選びました。この方法は総当たり法などとも呼ばれています。

説明変数が 2 つの場合は、組み合わせは 3 通り（height、diameter、height + diameter）だけ試せばよいですが、説明変数が増えると組み合わせ数は急激に増加します。そのため、このような場合には変数増加法や変数減少法、スパース回帰などの手法を使うことが一般的です。また、ランダムフォレストなどの機械学習アルゴリズムも変数選択に役立つ方法です。

In [ ]:
!rm acorns.clean.csv